In [1]:
import os
from urllib import request
import zipfile

from dataclasses import dataclass
from pathlib import Path
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories, get_size
from textSummarizer.logging import logger

In [2]:
os.chdir('../')


In [3]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [7]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        # Creates artifacts folder
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        data_ingestion_config = self.config.data_ingestion

        # Creates data_ingestion folder
        create_directories([data_ingestion_config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=data_ingestion_config.root_dir,
            source_URL=data_ingestion_config.source_URL,
            local_data_file=data_ingestion_config.local_data_file,
            unzip_dir=data_ingestion_config.unzip_dir
        )

        return data_ingestion_config


In [15]:
from pathlib import Path
import zipfile
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        """
        Download file from source URL and save it to local_data_file path.
        
        If file already exists, it will be skipped.
        """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f'{filename} downloaded successfully. Header is: \n {headers}')
        else:
            logger.info(f'File already exists. Size is: \n {get_size(Path(self.config.local_data_file))}')

    def extract_zip_file(self):
        """
        Extract zip file to unzip_dir path.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f'File {self.config.local_data_file} extracted to {self.config.unzip_dir}')

In [16]:
# Pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-06-03 17:09:32,543: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-06-03 17:09:32,545: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-03 17:09:32,546: INFO: common: created directory at: artifacts]
[2023-06-03 17:09:32,547: INFO: common: created directory at: artifacts/data_ingestion]
[2023-06-03 17:09:32,548: INFO: 2937009565: File already exists. Size is: 
 0.01]
[2023-06-03 17:09:32,650: INFO: 2937009565: File artifacts/data_ingestion/data.zip extracted to artifacts/data_ingestion]


'artifacts'